In [ ]:
!kaggle datasets download -d diraizel/anime-images-dataset

In [ ]:
import zipfile
with zipfile.ZipFile("/kaggle/working/anime-images-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("anime-images-dataset")

In [ ]:
import os
import shutil
from tqdm import tqdm  # Make sure to install tqdm if you haven't already

source_dir = r'/kaggle/working/anime-images-dataset/data/anime_images'
destination_dir = r'/kaggle/working/anime-images/images'

# Create destination folder if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Count total files for tqdm
all_files = []
for root, dirs, files in os.walk(source_dir):
    for file in files:
        all_files.append(os.path.join(root, file))

for file_path in tqdm(all_files, desc="Moving images"):
    file = os.path.basename(file_path)
    dest_path = os.path.join(destination_dir, file)
    
    # Handle file name conflicts by appending a counter
    counter = 1
    while os.path.exists(dest_path):
        base, ext = os.path.splitext(file)
        dest_path = os.path.join(destination_dir, f"{base}_{counter}{ext}")
        counter += 1

    shutil.copy2(file_path, dest_path)

print("✅ All images have been moved successfully.")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import torch.nn.parallel
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data_root="/kaggle/working/anime-images"

workers = 2

batch_size = 128

image_size = 64

n_channel = 3

n_z_noise = 100

n_feature_generator = 64

epochs = 10

lr = 0.002

beta1 = 0.5

ngpu = 0

In [ ]:
dataset=datasets.ImageFolder(root=data_root,
                            transform=transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5) , (0.5,0.5,0.5)),
                            ]))

dataloader=torch.utils.data.DataLoader(dataset , batch_size=batch_size,
                                       shuffle=True,num_workers=workers)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

real_batch=next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()

In [ ]:
from typing import Type
def init_weights(model : Type[nn.Module]) -> None :
    '''
    Function to Custom-Initialize the weights of the model
    according to the official research paper : arXiv:1511.06434v2

    Parameters : 
        mode : subclass of the torch.nn.Module , the model
    '''
    for m in model.modules():
        # If any of these layers occurs , the initialize the weights with 0.02 as std dev and 0.0 as mean
        if isinstance(m , (nn.Conv2d , nn.ConvTranspose2d , nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data , 0.0 , 0.02)  

In [ ]:
class Generator(nn.Module):
    def __init__(self,ngpu):
        '''
        Initialize the Generator class
        '''
        super(Generator , self).__init__()
        self.ngpu=ngpu
        # Generator architecture
        self.main=nn.Sequential(
            # input is Z , going into convolutional transpose
            nn.ConvTranspose2d(n_z_noise , n_feature_generator * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(n_feature_generator * 8),
            nn.ReLU(True),

            # state size : (ngf*8) x 4 x 4 -> (64*8) x 4 x 4 
            nn.ConvTranspose2d(n_feature_generator * 8, n_feature_generator * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_feature_generator * 4),
            nn.ReLU(True),

            # state size : (ngf*4) x 8 x 8 -> (64*4) x 8 x 8 
            nn.ConvTranspose2d( n_feature_generator * 4, n_feature_generator * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_feature_generator * 2),
            nn.ReLU(True),

            # state size : (ngf*2) x 16 x 16 -> (64*2) x 16 x 16 
            nn.ConvTranspose2d( n_feature_generator * 2, n_feature_generator, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_feature_generator),
            nn.ReLU(True), 
            
            # state size : (ngf) x 32 x 32 -> (64) x 32 x 32 
            nn.ConvTranspose2d( n_feature_generator, n_channel, 4, 2, 1, bias=False),
            nn.Tanh()

            # state size : n_channel x 64 x 64 -> 3 x 64 x 64
        )

    def forward(self, input : torch.Tensor) -> torch.Tensor :
        '''
        Forward pass for the generator

        Parameters : 
            input : A noise tensor of shape n_z_noise
        
        Returns : 
            torch.Tensor : a generated image of shape 3 x 64 x 64
        '''
        return self.main(input)

In [ ]:
gen_net=Generator(ngpu).to(device)

if (device.type=='cuda') and (ngpu > 1):
    gen_net=nn.DataParallel(gen_net , list(range(ngpu)))

gen_net.apply(init_weights)

print(gen_net)

In [ ]:
class Discriminator(nn.Module): 
    def __init__(self , ngpu):
        super(Discriminator , self).__init__()
        self.ngpu=ngpu

        self.main=nn.Sequential(

            nn.Conv2d(n_channel , n_feature_generator , 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2 , inplace=True),

            nn.Conv2d(n_feature_generator, n_feature_generator * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_feature_generator * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(n_feature_generator * 2, n_feature_generator * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_feature_generator * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(n_feature_generator * 4, n_feature_generator * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_feature_generator * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(n_feature_generator * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()           
        )

    def forward(self , input):
        return self.main(input)

In [ ]:
dis_net=Discriminator(ngpu).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    dis_net = nn.DataParallel(dis_net, list(range(ngpu)))

dis_net.apply(init_weights)

print(dis_net)

In [ ]:
criterion=nn.BCELoss()

fixed_input_noise = torch.randn(64, n_z_noise, 1, 1, device=device)

real_label = 1
fake_label = 0

optimizer_dis = optim.Adam(dis_net.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_gen = optim.Adam(gen_net.parameters(), lr=lr, betas=(beta1, 0.999))


In [ ]:
img_list=[]
gen_losses=[]
dis_losses=[]
iters=0

writer_real=SummaryWriter(f"logs/real")
writer_fake=SummaryWriter(f"logs/fake")


print("Starting Training Loop....")

for epoch in range(epochs):

    for i , data in enumerate(dataloader , 0):

        dis_net.zero_grad()

        real_to_cpu = data[0].to(device)
        b_size=real_to_cpu.size(0)
        label=torch.full((b_size,), real_label, dtype=torch.float, device=device)

        output=dis_net(real_to_cpu).view(-1)

        errorD_real = criterion(output, label)

        errorD_real.backward()
        D_x=output.mean().item()

        noise = torch.randn(b_size, n_z_noise, 1, 1, device=device)

        fake=gen_net(noise)
        label.fill_(fake_label)

        output = dis_net(fake.detach()).view(-1)

        errorD_fake = criterion(output, label)

        errorD_fake.backward()
        optimizer_dis.step()

        D_G_z1= output.mean().item()

        errorD=errorD_fake + errorD_real

        optimizer_dis.step()

        gen_net.zero_grad()
        label.fill_(real_label)

        output = dis_net(fake).view(-1)
        errorG = criterion(output, label)

        errorG.backward()
        D_G_z2 = output.mean().item()

        optimizer_gen.step()

        if i%50==0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, epochs, i, len(dataloader),
                     errorD.item(), errorG.item(), D_x, D_G_z1, D_G_z2))
            
            gen_losses.append(errorG.item())
            dis_losses.append(errorD.item())
            
            if (iters % 500 == 0) or ((epoch == epochs-1) and (i == len(dataloader)-1)):
                with torch.no_grad():
                    fake = gen_net(fixed_input_noise).detach().cpu()
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

            img_grid_real = vutils.make_grid(real_to_cpu[:32],normalize=True)
            img_grid_fake = vutils.make_grid(fake[:32],normalize=True)

            writer_real.add_image("Real", img_grid_real, global_step=iters)
            writer_fake.add_image("Fake", img_grid_fake, global_step=iters)

            iters += 1
        

# Save everything needed for resuming
torch.save({
    'epoch': epoch,
    'gen_state_dict': gen_net.state_dict(),
    'dis_state_dict': dis_net.state_dict(),
    'optimizer_gen': optimizer_gen.state_dict(),
    'optimizer_dis': optimizer_dis.state_dict(),
    'gen_losses': gen_losses,
    'dis_losses': dis_losses,
    'iters': iters
}, 'gan_checkpoint.pth')



In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(gen_losses,label="G")
plt.plot(dis_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())